<a href="https://colab.research.google.com/github/mrinalkokate/mrinalkokate/blob/main/ML_CA2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
pip install category_encoders

In [7]:
from pandas import read_csv, get_dummies,Series,DataFrame, to_datetime
from sklearn.linear_model import SGDRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from imblearn.over_sampling import SMOTE
from plotly import graph_objs
import plotly.figure_factory as ff
from category_encoders import TargetEncoder

data1=read_csv('/content/drive/MyDrive/Data Sets/Car Sales.xlsx - car_data 2.csv')# I am reading the data here using read_csv from Pandas

#data1.head()
#data1.shape
data1.info()
#data1.columns

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23906 entries, 0 to 23905
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Car_id         23906 non-null  object
 1   Date           23906 non-null  object
 2   Customer Name  23905 non-null  object
 3   Gender         23906 non-null  object
 4   Annual Income  23906 non-null  int64 
 5   Dealer_Name    23906 non-null  object
 6   Company        23906 non-null  object
 7   Model          23906 non-null  object
 8   Engine         23906 non-null  object
 9   Transmission   23906 non-null  object
 10  Color          23906 non-null  object
 11  Price ($)      23906 non-null  int64 
 12  Dealer_No      23906 non-null  object
 13  Body Style     23906 non-null  object
 14  Phone          23906 non-null  int64 
 15  Dealer_Region  23906 non-null  object
dtypes: int64(3), object(13)
memory usage: 2.9+ MB


In [8]:
data2 = data1.drop(["Car_id", "Dealer_Name", "Dealer_No ", "Phone", "Customer Name"], axis=1) # Feature Reeduction Technique USED- DROP MANUALLY

data2['Transmission']=data2['Transmission'].map({'Auto':1,'Manual':0}) #Binary Nature
data2['Gender']=data2['Gender'].map({'Male':1,'Female':0}) #Binary Nature
data2['Engine']=data2['Engine'].map({'DoubleÂ\xa0Overhead Camshaft':1,'Overhead Camshaft':0}) #Binary Nature
data2

,Date,Gender,Annual Income,Company,Model,Engine,Transmission,Color,Price ($),Body Style,Dealer_Region
0,1/2/2022,1,13500,Ford,Expedition,1,1,Black,26000,SUV,Middletown
1,1/2/2022,1,1480000,Dodge,Durango,1,1,Black,19000,SUV,Aurora
2,1/2/2022,1,1035000,Cadillac,Eldorado,0,0,Red,31500,Passenger,Greenville
3,1/2/2022,1,13500,Toyota,Celica,0,0,Pale White,14000,SUV,Pasco
4,1/2/2022,1,1465000,Acura,TL,1,1,Red,24500,Hatchback,Janesville
...,...,...,...,...,...,...,...,...,...,...,...
23901,12/31/2023,1,13500,Plymouth,Voyager,0,0,Red,12000,Passenger,Pasco
23902,12/31/2023,0,900000,Chevrolet,Prizm,1,1,Black,16000,Hardtop,Middletown
23903,12/31/2023,1,705000,BMW,328i,0,0,Red,21000,Sedan,Scottsdale
23904,12/31/2023,1,13500,Chevrolet,Metro,1,1,Black,31000,Passenger,Austin


In [9]:
#date conversion
data2.shape
to_datetime(392175900  , unit='s')
DataFrame(data2['Date'])
data2['Date']=to_datetime(data2['Date'],format='mixed') #format='%d-%m-%y %H:%M:%S'
data2.info()
DataFrame(data2['Date'])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23906 entries, 0 to 23905
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Date           23906 non-null  datetime64[ns]
 1   Gender         23906 non-null  int64         
 2   Annual Income  23906 non-null  int64         
 3   Company        23906 non-null  object        
 4   Model          23906 non-null  object        
 5   Engine         23906 non-null  int64         
 6   Transmission   23906 non-null  int64         
 7   Color          23906 non-null  object        
 8   Price ($)      23906 non-null  int64         
 9   Body Style     23906 non-null  object        
 10  Dealer_Region  23906 non-null  object        
dtypes: datetime64[ns](1), int64(5), object(5)
memory usage: 2.0+ MB


,Date
0,2022-01-02
1,2022-01-02
2,2022-01-02
3,2022-01-02
4,2022-01-02
...,...
23901,2023-12-31
23902,2023-12-31
23903,2023-12-31
23904,2023-12-31


In [10]:
data2['year'] = data2['Date'].dt.year #DATE CONVERSION
data2['month'] = data2['Date'].dt.month
data2['day'] = data2['Date'].dt.day
data2['Day_week'] = data2['Date'].dt.dayofweek

data2.info()
data2.tail()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23906 entries, 0 to 23905
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Date           23906 non-null  datetime64[ns]
 1   Gender         23906 non-null  int64         
 2   Annual Income  23906 non-null  int64         
 3   Company        23906 non-null  object        
 4   Model          23906 non-null  object        
 5   Engine         23906 non-null  int64         
 6   Transmission   23906 non-null  int64         
 7   Color          23906 non-null  object        
 8   Price ($)      23906 non-null  int64         
 9   Body Style     23906 non-null  object        
 10  Dealer_Region  23906 non-null  object        
 11  year           23906 non-null  int32         
 12  month          23906 non-null  int32         
 13  day            23906 non-null  int32         
 14  Day_week       23906 non-null  int32         
dtypes: datetime64[ns](1

,Date,Gender,Annual Income,Company,Model,Engine,Transmission,Color,Price ($),Body Style,Dealer_Region,year,month,day,Day_week
23901,2023-12-31,1,13500,Plymouth,Voyager,0,0,Red,12000,Passenger,Pasco,2023,12,31,6
23902,2023-12-31,0,900000,Chevrolet,Prizm,1,1,Black,16000,Hardtop,Middletown,2023,12,31,6
23903,2023-12-31,1,705000,BMW,328i,0,0,Red,21000,Sedan,Scottsdale,2023,12,31,6
23904,2023-12-31,1,13500,Chevrolet,Metro,1,1,Black,31000,Passenger,Austin,2023,12,31,6
23905,2023-12-31,1,1225000,Lexus,ES300,1,1,Pale White,27500,Hardtop,Middletown,2023,12,31,6


In [11]:
data2=data2.drop(['Date'], axis=1)  #DATE FEATURE REMOVED
data2.info()
data2.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23906 entries, 0 to 23905
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Gender         23906 non-null  int64 
 1   Annual Income  23906 non-null  int64 
 2   Company        23906 non-null  object
 3   Model          23906 non-null  object
 4   Engine         23906 non-null  int64 
 5   Transmission   23906 non-null  int64 
 6   Color          23906 non-null  object
 7   Price ($)      23906 non-null  int64 
 8   Body Style     23906 non-null  object
 9   Dealer_Region  23906 non-null  object
 10  year           23906 non-null  int32 
 11  month          23906 non-null  int32 
 12  day            23906 non-null  int32 
 13  Day_week       23906 non-null  int32 
dtypes: int32(4), int64(5), object(5)
memory usage: 2.2+ MB


,Gender,Annual Income,Company,Model,Engine,Transmission,Color,Price ($),Body Style,Dealer_Region,year,month,day,Day_week
0,1,13500,Ford,Expedition,1,1,Black,26000,SUV,Middletown,2022,1,2,6
1,1,1480000,Dodge,Durango,1,1,Black,19000,SUV,Aurora,2022,1,2,6
2,1,1035000,Cadillac,Eldorado,0,0,Red,31500,Passenger,Greenville,2022,1,2,6
3,1,13500,Toyota,Celica,0,0,Pale White,14000,SUV,Pasco,2022,1,2,6
4,1,1465000,Acura,TL,1,1,Red,24500,Hatchback,Janesville,2022,1,2,6


In [12]:
print(data2['Company'].unique())
print(data2['Model'].unique())
print(data2['Body Style'].unique())
print(data2['Dealer_Region'].unique())
print(data2['Color'].unique())

data2['Body Style'] = data2['Body Style'].map({'SUV':1, 'Passenger':2, 'Hatchback':3, 'Hardtop':4, 'Sedan':5})
data2['Color'] = data2['Color'].map({'Black':1, 'Red':2, 'Pale White':3})
data2['Dealer_Region'] = data2['Dealer_Region'].map({'Middletown':1, 'Aurora':2, 'Greenville':3, 'Pasco':4, 'Janesville':5, 'Scottsdale':6, 'Austin':7})
data2.head()

#data3 = get_dummies(data2,('Company','Color','Body Style','Dealer_Region')) #ONE-HOT ENCODING
#data3.head()

['Ford' 'Dodge' 'Cadillac' 'Toyota' 'Acura' 'Mitsubishi' 'Chevrolet'
 'Nissan' 'Mercury' 'BMW' 'Chrysler' 'Subaru' 'Hyundai' 'Honda' 'Infiniti'
 'Audi' 'Porsche' 'Volkswagen' 'Buick' 'Saturn' 'Mercedes-B' 'Jaguar'
 'Volvo' 'Pontiac' 'Lincoln' 'Oldsmobile' 'Lexus' 'Plymouth' 'Saab' 'Jeep']
['Expedition' 'Durango' 'Eldorado' 'Celica' 'TL' 'Diamante' 'Corolla'
 'Galant' 'Malibu' 'Escort' 'RL' 'Pathfinder' 'Grand Marquis' '323i'
 'Sebring Coupe' 'Forester' 'Accent' 'Land Cruiser' 'Accord' '4Runner'
 'I30' 'A4' 'Carrera Cabrio' 'Jetta' 'Viper' 'Regal' 'LHS' 'LW' '3000GT'
 'SLK230' 'Civic' 'S-Type' 'S40' 'Mountaineer' 'Park Avenue'
 'Montero Sport' 'Sentra' 'S80' 'Lumina' 'Bonneville' 'C-Class' 'Altima'
 'DeVille' 'Stratus' 'Cougar' 'SW' 'C70' 'SLK' 'Tacoma' 'M-Class' 'A6'
 'Intrepid' 'Sienna' 'Eclipse' 'Contour' 'Town car' 'Focus' 'Mustang'
 'Cutlass' 'Corvette' 'Impala' 'Cabrio' 'Dakota' '300M' '328i' 'Bravada'
 'Maxima' 'Ram Pickup' 'Concorde' 'V70' 'Quest' 'ES300' 'SL-Class'
 'Explorer' 

,Gender,Annual Income,Company,Model,Engine,Transmission,Color,Price ($),Body Style,Dealer_Region,year,month,day,Day_week
0,1,13500,Ford,Expedition,1,1,1,26000,1,1,2022,1,2,6
1,1,1480000,Dodge,Durango,1,1,1,19000,1,2,2022,1,2,6
2,1,1035000,Cadillac,Eldorado,0,0,2,31500,2,3,2022,1,2,6
3,1,13500,Toyota,Celica,0,0,3,14000,1,4,2022,1,2,6
4,1,1465000,Acura,TL,1,1,2,24500,3,5,2022,1,2,6


In [13]:
X = data2.drop(['Price ($)'], axis = 1) #FEATURES
Y = data2['Price ($)'] #LABEL
print(X.shape)
print(Y.shape)

(23906, 13)
(23906,)


In [14]:
Target_Encoder1 = TargetEncoder(cols=['Company']).fit(X,Y)#TARGET ENCODING
Target_Encoder2 = TargetEncoder(cols=['Model']).fit(X,Y)
X = Target_Encoder1.transform(X)
X = Target_Encoder2.transform(X)
print(X.info())
X.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23906 entries, 0 to 23905
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Gender         23906 non-null  int64  
 1   Annual Income  23906 non-null  int64  
 2   Company        23906 non-null  float64
 3   Model          23906 non-null  float64
 4   Engine         23906 non-null  int64  
 5   Transmission   23906 non-null  int64  
 6   Color          23906 non-null  int64  
 7   Body Style     23906 non-null  int64  
 8   Dealer_Region  23906 non-null  int64  
 9   year           23906 non-null  int32  
 10  month          23906 non-null  int32  
 11  day            23906 non-null  int32  
 12  Day_week       23906 non-null  int32  
dtypes: float64(2), int32(4), int64(7)
memory usage: 2.0 MB
None


,Gender,Annual Income,Company,Model,Engine,Transmission,Color,Body Style,Dealer_Region,year,month,day,Day_week
0,1,13500,29263.682156,22503.488391,1,1,1,1,1,2022,1,2,6
1,1,1480000,26406.341113,22436.851145,1,1,1,1,2,2022,1,2,6
2,1,1035000,40972.093558,41919.857750,0,0,2,2,3,2022,1,2,6
3,1,13500,29513.120721,25755.326164,0,0,3,1,4,2022,1,2,6
4,1,1465000,24758.561684,22821.631970,1,1,2,3,5,2022,1,2,6


In [15]:
corrs = X.corr()
figure = ff.create_annotated_heatmap(corrs.values,list(corrs.columns),list(corrs.columns), corrs.round(2).values,showscale=True)
figure.show()

In [16]:
X1=X.drop(['Transmission'],axis=1) #FEATURE REDUCTION VIA HEAT MAP
X1.info()
Y.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23906 entries, 0 to 23905
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Gender         23906 non-null  int64  
 1   Annual Income  23906 non-null  int64  
 2   Company        23906 non-null  float64
 3   Model          23906 non-null  float64
 4   Engine         23906 non-null  int64  
 5   Color          23906 non-null  int64  
 6   Body Style     23906 non-null  int64  
 7   Dealer_Region  23906 non-null  int64  
 8   year           23906 non-null  int32  
 9   month          23906 non-null  int32  
 10  day            23906 non-null  int32  
 11  Day_week       23906 non-null  int32  
dtypes: float64(2), int32(4), int64(6)
memory usage: 1.8 MB
<class 'pandas.core.series.Series'>
RangeIndex: 23906 entries, 0 to 23905
Series name: Price ($)
Non-Null Count  Dtype
--------------  -----
23906 non-null  int64
dtypes: int64(1)
memory usage: 186.9 KB


In [17]:
# Linear Regression (LR)

LinearRegression1 = SGDRegressor(random_state = 1, penalty = None) # building
Hparameter1 = {'eta0': [.0001, .001, .01, .1, 1], 'max_iter':[10000, 20000, 30000, 40000]}
grid_search1 = GridSearchCV(estimator=LinearRegression1, param_grid=Hparameter1, scoring='r2', cv=5)
grid_search1.fit(X1, Y)

# results = DataFrame.from_dict(grid_search1.cv_results_)
# print("Cross-validation results:\n", results)
best_parameters = grid_search1.best_params_
print("Best parameters: ", best_parameters)
best_result = grid_search1.best_score_
print("Best result: ", best_result)
best_model = grid_search1.best_estimator_
print("Intercept β0: ", best_model.intercept_)
#print(DataFrame(zip(X1.columns, best_model.coef_), columns=['Features','Coefficients']))
print(DataFrame(zip(X1.columns, best_model.coef_), columns=['Features','Coefficients']))

Best parameters:  {'eta0': 0.0001, 'max_iter': 10000}
Best result:  -3.435873970522946e+28
Intercept β0:  [22373812.58825389]
         Features  Coefficients
0          Gender -1.191945e+09
1   Annual Income -1.232075e+11
2         Company  2.620001e+11
3           Model  1.114577e+12
4          Engine  1.936228e+08
5           Color  5.236332e+09
6      Body Style  7.744496e+09
7   Dealer_Region -1.809536e+10
8            year  4.527401e+10
9           month  3.368089e+09
10            day -8.332846e+10
11       Day_week -1.581133e+10


In [ ]:
#  Regularization
LinearRegression = SGDRegressor(random_state = 1, penalty = 'elasticnet')
parameter = {'eta0': [.0001, .001, .01, .1, 1], 'max_iter':[10000, 20000, 30000, 40000],'alpha': [.001, .01, .1, 1,10, 100], 'l1_ratio': [0,0.25,0.5,0.75,1]}

grid_search = GridSearchCV(estimator=LinearRegression, param_grid=parameter, scoring='r2', cv=5)
grid_search.fit(X1, Y)

best_parameters = grid_search.best_params_
print("Best parameters: ", best_parameters)
best_result = grid_search.best_score_
print("Best result: ", best_result)
best_model = grid_search.best_estimator_
print("Intercept β0: ", best_model.intercept_)
print(DataFrame(zip(X1.columns, best_model.coef_), columns=['Features','Coefficients']))

In [ ]:
#Modified mean square error
r, c=X1.shape
print(X1.shape)
print(r,c)
print(4/5*r,c)
modified_r2 = 1-(1-best_result)*(4/5*r-1)/(4/5*r-c-1)# 4/5*r is number of rows in training set, c is number of columns
print("modified_r2: ", modified_r2)

In [ ]:
#Support Vector Regression
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV
SVRegressor = SVR()
cv=8
parameters = {'kernel':['linear','poly','rbf','sigmoid'], 'C': [100,1000,10000], 'epsilon': [100,1000,10000]}
grid_search1 = GridSearchCV(estimator=SVRegressor, param_grid=parameters, scoring='r2')
grid_search1.fit(X1, Y)

best_parameters = grid_search1.best_params_
print("Best parameters: ", best_parameters)
best_result = grid_search1.best_score_
print("Best result: ", best_result)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from pandas import Series
RF_Regressor = RandomForestRegressor(criterion='squared_error', max_features='sqrt', random_state=1)
no_Trees = {'n_estimators': [10,20,30,40,50,100]}
r, c=X.shape
print(r,c)
grid_search3 = GridSearchCV(estimator=RF_Regressor, param_grid=no_Trees, scoring='r2', cv=5)
grid_search3.fit(X, Y)

best_parameters = grid_search3.best_params_
print("Best parameters: ", best_parameters)
best_result = grid_search3.best_score_
print("best_score: ", best_result)
modified_r2 = 1-(1-best_result)*(4/5*r-1)/(4/5*r-c-1)
print("modified_r2: ", modified_r2)
Important_feature = Series(grid_search3.best_estimator_.feature_importances_, index=list(X)).sort_values(ascending=False) # Getting feature importances list for the best model
print(Important_feature)